In [378]:
import numpy as np
from py3gpp import *
import matplotlib.pyplot as plt

PDS
nrPDSCHConfig()
nrPDSCHDMRS()

scs = 30e3
numRB = 273
numSub = numRB * 12
fft_size = int(2**np.ceil(np.log2(numSub)))
Fs = int(scs * fft_size)
Qm = 2
N = Qm * numSub
NP = int(N * 0.25)


pilotsBits = nrPRBS(0, N)
symbols = nrSymbolModulate(pilotsBits, 'qpsk')
#dataBits = np.random.randint(0, 2, N)
#symbols = nrSymbolModulate(dataBits, 'qpsk')
#pilotIndex = np.arange(0, fft_size, int(fft_size/len(pilotsSymbols)))
#symbols[pilotIndex] = pilotsSymbols


grid = 1j*np.zeros([fft_size, 5])
grid[int(fft_size/2 - numSub/2):int(fft_size/2 + numSub/2), 1] = symbols
plt.imshow(abs(grid), extent=[0, 5, 0, 40])
txSignal, info = nrOFDMModulate(grid=grid, initialNSlot=1, Nfft=fft_size, SampleRate=Fs, scs=30)
plt.figure()
plt.plot(abs(txSignal))



In [446]:
import pandas as pd


csv = pd.read_csv("C:\\Users\\iranj\\Documents\\TDL-A.csv")
amplitudes = csv['amplitudes']
delays = csv['delays']

In [447]:
desiredDelay = 300
#delays = np.array([0, 5.133, 5.440, 5630, 5.440, 7.112, 19.092, 19.293, 19.589, 26.426, 37.136, 54.524, 120.034, 206.519]) * desiredDelay
#amplitudes = np.array([-0.03, -15.8, -18.1, -19.8, -22.9, -22.4, -18.6, -20.8, -22.6, -22.3, -25.6, -20.2, -29.8, -29.2])


#delays = np.array([0, 12, 40, 100])
#amplitudes = np.array([-4, -0, -2, -6])
delays = delays * desiredDelay
delays  = np.int16(delays / (1e9/Fs))

plt.figure()
plt.stem(delays, amplitudes, bottom=-50)


<StemContainer object of 3 artists>

In [448]:
h = np.zeros(max(delays)+1)
#delays = np.array([0, 10, 150, 180, 230, 320, ])
h[delays] = 10**(np.array(amplitudes)/10)
H = np.fft.fft(h, fft_size)

In [449]:
%matplotlib qt
rxSignal = np.convolve(txSignal, h)

dataRx = nrOFDMDemodulate(waveform=rxSignal, initialNSlot=1, SampleRate=Fs, scs=30, Nfft=fft_size, nrb=numRB)

pilotIndex = np.arange(0, len(dataRx), 2)
H = dataRx[pilotIndex, 1] * symbols[pilotIndex].conj().T

H = np.interp(np.arange(0, len(dataRx), pilotIndex, H)

h = np.fft.ifftshift(np.fft.ifft(H, fft_size))

#hh = 10*np.log10(abs(h))
#hhh = -np.var(hh)*np.ones(fft_size)
#hhh[np.arange(1, 2*fft_size, 2)] = hh 


#idx = np.argmax(hh)
#plt.figure()
#plt.plot(hh[int(fft_size/2)-1:], '-')



In [450]:
out = np.correlate(rxSignal, txSignal[info['CyclicPrefixLengths'][1]:], 'valid')
plt.figure()
plt.plot(10*np.log10(abs(out)), '-')

In [451]:
plt.figure()
plt.plot(abs(h[int(len(h)/2)-1:]))

In [452]:
import numpy as np

class MultiArmedBandit:
    def __init__(self, A):
        self.A = A
        self.epsilon = 0.2
        self.N = np.zeros(A)
        self.Q = np.zeros(A)
        

    def select_action(self):
        if np.random.rand() > self.epsilon:
            return np.argmax(self.Q)
        else:
            return np.random.randint(0, self.A)
    
    def get_reward(self):
        np.random.rand()
        return 
        
    def run_step(self):
        a = self.select_action()
        re 


    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        new_value = ((n - 1) / n) * value + (1 / n) * reward
        self.values[chosen_arm] = new_value
        

# Exemplo de uso
n_arms = 5
epsilon = 0.1
bandit = MultiArmedBandit(n_arms, epsilon)

n_steps = 10000
rewards = np.random.randn(n_steps, n_arms)  # Recompensas simuladas para cada braço e passo
total_reward = 0

for step in range(n_steps):
    chosen_arm = bandit.select_arm()
    reward = rewards[step, chosen_arm]
    bandit.update(chosen_arm, reward)
    total_reward += reward

print(f"Total reward: {total_reward}")
print(f"Counts: {bandit.counts}")
print(f"values: {bandit.values}")




TypeError: MultiArmedBandit.__init__() takes 2 positional arguments but 3 were given